In [ ]:
from tespy.components import PolynomialCompressor, Source, Sink
from tespy.connections import Connection
from tespy.networks import Network


class PolynomialCompressorWithCooling(PolynomialCompressor):

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [ ]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=50)

b1.set_attr(fluid={"water": 1}, m=0.5, T=15, p=1)
b2.set_attr(T=25, p=1)

compressor.set_attr(Q_diss_rel=0.1)

nw.solve("design")

- the base component class has pairwise mass flow and fluid composition balance:
  - in1 matches with out1
  - in2 matches with out2
  - ...
- this is automatically expanded with every new pair of ports
- if you want to add ports with other names or non-paired ports, this may break
- in this case we can verify, if things went the right way by checking mass flow
  and fluid composition in results

In [ ]:
nw.print_results()

In [ ]:
b1.fluid.val, b2.fluid.val

In [ ]:
from tespy.tools.data_containers import ComponentMandatoryConstraints as dc_cmc


class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.Q_diss_rel.is_set:
            self.Q_diss_rel.is_set = True
            self.Q_diss_rel.val = 0
            self.Q_diss_rel.val_SI = 0
        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def cooling_energy_balance_func(self):
        usable_share = 0.9
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.Q_diss_rel.val_SI)
                + self.inl[0].h.val_SI * (self.Q_diss_rel.val_SI / (1 - self.Q_diss_rel.val_SI))
            ) * usable_share
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [ ]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=50)

b1.set_attr(fluid={"water": 1}, m=1, T=15, p=1)
b2.set_attr(T=25, p=1)
compressor.set_attr(Q_diss_rel=0.1)

nw.solve("design")

- we now have one extra equation, that means we can set one less parameter,
  e.g. cooling mass flow

In [ ]:
b1.set_attr(m=None)
nw.solve("design")

In [ ]:
b1.m.val_SI

In [ ]:
from tespy.tools.data_containers import ComponentProperties as dc_cp
from tespy.tools.helpers import TESPyComponentError


class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.eta_recovery.is_set:
            msg = (
                f"The component {self.label} of type {self.__class__.__name__}"
                "requires you to specify the share of heat recovery "
                "eta_recovery."
            )
            raise TESPyComponentError(msg)

        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def get_parameters(self):
        params = super().get_parameters()
        params["eta_recovery"] = dc_cp()
        return params

    def cooling_energy_balance_func(self):
        usable_share = self.eta_recovery.val_SI
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.Q_diss_rel.val_SI)
                + self.inl[0].h.val_SI * (self.Q_diss_rel.val_SI / (1 - self.Q_diss_rel.val_SI))
            ) * usable_share
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [ ]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=25)

b1.set_attr(fluid={"water": 1}, T=15, p=1)
b2.set_attr(T=25, p=1)
compressor.set_attr(Q_diss_rel=0.1, eta_recovery=0.9)

nw.solve("design")

In [ ]:
compressor.Q_diss.val

In [ ]:
b1.m.val_SI * (b2.h.val_SI - b1.h.val_SI)

In [ ]:
b1.set_attr(m=0.005)
b2.set_attr(T=None)
nw.solve("design")

In [ ]:
b2.T.val, c2.T.val

In [ ]:
h_2 = c1.h.val_SI + (c2.h.val_SI - c1.h.val_SI) / (1 - compressor.Q_diss_rel.val_SI)
c2.p.val_SI

In [ ]:
from tespy.tools.fluid_properties import T_mix_ph


T_mix_ph(c2.p.val_SI, h_2, c2.fluid_data) - 273.15

In [ ]:
compressor.eta_s.val

In [ ]:
class PolynomialCompressorWithCooling(PolynomialCompressor):

    def _preprocess(self, row_idx):
        if not self.eta_recovery.is_set:
            msg = (
                f"The component {self.label} of type {self.__class__.__name__}"
                "requires you to specify the share of heat recovery "
                "eta_recovery."
            )
            raise TESPyComponentError(msg)

        return super()._preprocess(row_idx)

    @staticmethod
    def inlets():
        return ['in1', 'in2']

    @staticmethod
    def outlets():
        return ['out1', 'out2']

    def get_mandatory_constraints(self) -> dict:
        constraints = super().get_mandatory_constraints()
        # this is a dictionary
        constraints["cooling_energy_balance_constraints"] = dc_cmc(
            func=self.cooling_energy_balance_func,
            dependents=self.cooling_energy_balance_dependents,
            num_eq_sets=1
        )
        return constraints

    def get_parameters(self):
        params = super().get_parameters()
        params["eta_recovery"] = dc_cp()
        params["td_minimal"] = dc_cp(
            min_val=0
        )
        return params

    def cooling_energy_balance_func(self):
        usable_share = self.eta_recovery.val_SI
        residual = (
            self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)
            + self.inl[0].m.val_SI * (
                self.outl[0].h.val_SI
                - self.outl[0].h.val_SI / (1 - self.Q_diss_rel.val_SI)
                + self.inl[0].h.val_SI * (self.Q_diss_rel.val_SI / (1 - self.Q_diss_rel.val_SI))
            ) * usable_share
        )
        return residual

    def cooling_energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[1].m,
            self.inl[0].h, self.inl[1].h,
            self.outl[0].h, self.outl[1].h
        ]

    def calc_parameters(self):
        super().calc_parameters()

        i = self.inl[0]
        o = self.outl[0]
        h_2 = (
            (o.h.val_SI - i.h.val_SI * self.Q_diss_rel.val_SI)
            / (1 - self.Q_diss_rel.val_SI)
        )

        T_max_compressor_internal = T_mix_ph(
            self.outl[0].p.val_SI,
            h_2,
            self.outl[0].fluid_data,
            self.outl[0].mixing_rule,
            T0=self.outl[0].T.val_SI
        )
        self.td_minimal.val_SI = (
            T_max_compressor_internal
            - self.outl[1].T.val_SI
        )


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)


gas_inlet = Source("gas inlet")
gas_outlet = Sink("gas outlet")

water_inlet = Source("water cold")
water_outlet = Sink("water hot")

compressor = PolynomialCompressorWithCooling("compressor")

c1 = Connection(gas_inlet, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", gas_outlet, "in1", label="c2")

b1 = Connection(water_inlet, "out1", compressor, "in2", label="b1")
b2 = Connection(compressor, "out2", water_outlet, "in1", label="b2")


nw.add_conns(c1, c2, b1, b2)

In [ ]:
c1.set_attr(fluid={"R290": 1}, m=1, T_dew=10, td_dew=10)
c2.set_attr(T_dew=60, td_dew=25)

b1.set_attr(fluid={"water": 1}, T=15, m=0.05, p=1)
b2.set_attr(p=1)
compressor.set_attr(Q_diss_rel=0.1, eta_recovery=0.9)

nw.solve("design")